## RAG로 AI소믈리에 wine pairing

In [1]:
from dotenv import load_dotenv
import os

load_dotenv(override=True, dotenv_path="../.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")

In [2]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_openai import ChatOpenAI

## LLM을 통한 요리 이미지 -> 맛과 풍미 (image to text)

In [ ]:
# image 에서 맛(풍미)에 대한 설명 text 생성
def describe_dish_flavor(query):
    # pass
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
            Persona: You are a highly skilled and perceptive culinary expert with a deep understanding of flavors, aromas, and textures in a wide variety of cuisines. Your personality is professional, insightful, and approachable, dedicated to helping users understand and appreciate the complexities of taste in their food experiences. You are passionate about exploring subtle nuances in ingredients and dishes, making flavor analysis accessible and engaging.

            Role: Your role is to guide users in understanding and analyzing the taste, aroma, texture, and overall flavor profile of various foods. You provide detailed descriptions of flavors and offer insights into how different ingredients, cooking techniques, and seasonings influence the dish's final taste. You also help users make informed choices about ingredient combinations and cooking methods to achieve desired flavors in their culinary creations.
            Examples:
            Flavor Profile Analysis: If a user describes a dish with grilled lamb seasoned with rosemary and garlic, you might explain how the earthy, woody notes of rosemary enhance the rich, savory flavor of the lamb. You could also describe how the caramelization from grilling adds a layer of smokiness, balanced by the mild sweetness of roasted garlic.
            Texture and Mouthfeel Explanation: If a user is tasting a creamy mushroom risotto, you might highlight the importance of the dish’s creamy, velvety texture achieved through the slow release of starch from Arborio rice. You could also mention how the umami-rich flavor of mushrooms adds depth to the dish, while the cheese provides a slight saltiness that balances the creaminess.
            Pairing Suggestions: If a user is preparing a spicy Thai green curry, you could recommend balancing its heat with a slightly sweet or acidic side, such as a cucumber salad or coconut rice. You might explain how the coolness of cucumber contrasts with the curry’s heat, and how the subtle sweetness in coconut rice tempers the dish’s spiciness, creating a harmonious dining experience.
         """),
        ("human", """
            이미지의 요리명과 풍미를 한 문장으로 요약해 주세요. 영어로 표현해 주세요.
        """)
    ])

    # image url list
    template = []

    if query.get("image_urls"):
        template += [{"image_url": image_url} for image_url in query["image_urls"]]

    prompt += HumanMessagePromptTemplate.from_template(template)

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens=4095)

    chain = prompt | llm | StrOutputParser()

    return chain

In [4]:
from langchain_core.runnables import RunnableLambda

r1 = RunnableLambda(describe_dish_flavor)

In [5]:
url = 'https://sitem.ssgcdn.com/95/55/96/item/1000346965595_i1_750.jpg'
res = r1.invoke({
    "image_urls": [url]
})
print(res)

This dish features grilled pork belly, known for its rich, savory flavor and crispy texture, served alongside spicy and tangy kimchi for a delightful contrast.


- prompt : 전문적인 요리 사진을 얻을 수 있는 웹사이트를 알려줘

## 사용자 프롬프트 vector화, 유사도 높은 top=5 찾기

In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
import os

embedding = OpenAIEmbeddings(model=os.getenv("OPENAI_EMBEDDING_MODEL"))
vector_store = PineconeVectorStore(
    index_name=os.getenv("PINECONE_INDEX_NAME"),
    embedding=embedding,
    namespace=os.getenv("PINECONE_NAMESPACE")
)

# query = "이 요리는 풍부하고 고소한 맛과 바삭한 식감이 특징인 구운 삼겹살에, 매콤하고 새콤한 김치를 곁들여 맛의 대비를 즐길 수 있도록 구성된 메뉴입니다."
query = "This dish features grilled pork belly, known for its rich, savory flavor and crispy texture, served alongside spicy and tangy kimchi for a delightful contrast."
vector_store.similarity_search(query, namespace=os.getenv("PINECONE_NAMESPACE"), k=5)

[Document(id='b25d3b2a-9680-43c9-8135-bd0aaff51552', metadata={'row': 29260, 'source': './wine_reviews/winemag-data-130k-v2.csv'}, page_content=": 29260\ncountry: Germany\ndescription: Tantalizing spice notes mix with aromas of quince and lanolin on this fabulous bargain Riesling. While intensely tropical on the palate, it's remarkably nuanced with hints of potpourri, white flowers, and a sharp, steely acidity. Perfect as an aperitif, or paired with spicy, herbaceous Asian cuisine.\ndesignation: Kabinett\npoints: 90\nprice: 13.0\nprovince: Mosel\nregion_1: \nregion_2: \ntaster_name: Anna Lee C. Iijima\ntaster_twitter_handle: \ntitle: Karl Kaspar 2010 Kabinett Riesling (Mosel)\nvariety: Riesling\nwinery: Karl Kaspar"),
 Document(id='4264f041-e1d7-4ab8-9ea8-8a8a2180c2a4', metadata={'row': 45075, 'source': './wine_reviews/winemag-data-130k-v2.csv'}, page_content=": 45075\ncountry: New Zealand\ndescription: A mouthfilling, off-dry Pinot Gris, Kim Crawford's 2007 continues his success with 

In [7]:
def search_wines(dish_flavor):
    embedding = OpenAIEmbeddings(model=os.getenv("OPENAI_EMBEDDING_MODEL"))
    vector_store = PineconeVectorStore(
        index_name=os.getenv("PINECONE_INDEX_NAME"),
        embedding=embedding,
        namespace=os.getenv("PINECONE_NAMESPACE")
    )

    results =  vector_store.similarity_search(dish_flavor, namespace=os.getenv("PINECONE_NAMESPACE"), k=5)

    return {
        "dish_flavor": dish_flavor,
        "wine_reviews": "\n".join([doc.page_content for doc in results])
    }

# query = "이 요리는 삼겹살과 김치로, 고소하고 육즙이 풍부한 삼겹살의 풍미가 매콤하고 새콤한 김치와 어우러져 조화로운 맛을 냅니다."

# results = search_wines(query)

# print(results.dish_flavor)
# print(results.wine_reviews)

## RunnableLambda로 실행

In [8]:
r2 = RunnableLambda(search_wines)
query = "This dish is made with pork belly and kimchi, and the rich, juicy flavor of the pork belly combines with the spicy, sour flavor of the kimchi to create a harmonious taste."
# query = "이 요리는 삼겹살과 김치로, 고소하고 육즙이 풍부한 삼겹살의 풍미가 매콤하고 새콤한 김치와 어우러져 조화로운 맛을 냅니다."
res = r2.invoke(query)
print(res.get("dish_flavor"))
print(res.get("wine_reviews"))

This dish is made with pork belly and kimchi, and the rich, juicy flavor of the pork belly combines with the spicy, sour flavor of the kimchi to create a harmonious taste.
: 29260
country: Germany
description: Tantalizing spice notes mix with aromas of quince and lanolin on this fabulous bargain Riesling. While intensely tropical on the palate, it's remarkably nuanced with hints of potpourri, white flowers, and a sharp, steely acidity. Perfect as an aperitif, or paired with spicy, herbaceous Asian cuisine.
designation: Kabinett
points: 90
price: 13.0
province: Mosel
region_1: 
region_2: 
taster_name: Anna Lee C. Iijima
taster_twitter_handle: 
title: Karl Kaspar 2010 Kabinett Riesling (Mosel)
variety: Riesling
winery: Karl Kaspar
: 39669
country: Germany
description: Rich, ripe bramble and berry notes penetrate deeply in this silky, supple Pinot Noir. It's intensely concentrated, but brightened by a shimmering streak of acidity and fresh herbal tones. A complex, impeccably balanced wine

## r1, r2로 구성

In [12]:
img_url = 'https://sitem.ssgcdn.com/95/55/96/item/1000346965595_i1_750.jpg'

chain = r1 | r2
res = chain.invoke({
    "image_urls": [img_url]
})

In [13]:
print(res.get("dish_flavor"))
print(res.get("wine_reviews"))

This dish features succulent grilled pork belly, characterized by its rich, savory flavor and crispy texture, complemented by the tangy and spicy notes of kimchi.
: 45075
country: New Zealand
description: A mouthfilling, off-dry Pinot Gris, Kim Crawford's 2007 continues his success with this variety. Pear and citrus notes finish clean, a refreshing blend of sugar and acid. Drink now.
designation: 
points: 87
price: 17.0
province: Marlborough
region_1: 
region_2: 
taster_name: Joe Czerwinski
taster_twitter_handle: @JoeCz
title: Kim Crawford 2007 Pinot Gris (Marlborough)
variety: Pinot Gris
winery: Kim Crawford
: 2216
country: New Zealand
description: This is round and mouthfilling without being fat or blowsy, finishing fairly crisp and dry. Along the way, hints of apple and citrus alternate with almond skin and cinnamon, providing a delicate balance of fruity and savory.
designation: 
points: 88
price: 17.0
province: Marlborough
region_1: 
region_2: 
taster_name: Joe Czerwinski
taster_t

## 와인 추천

In [ ]:
def recommand_wines(query):
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
            🍷 Wine Sommelier System Prompt
            Persona

            You are an expert wine sommelier with professional experience in restaurants and fine dining environments.
            You specialize in food and wine pairing and focus on helping users choose wines that enhance their overall dining experience.
            You think like a human sommelier: balancing sensory logic, customer preferences, and situational context rather than following rigid rules.

            Role
            Your role is to recommend optimal wine pairings based on:
            - The structure of the food (fat, acidity, saltiness, sweetness, cooking method)
            - The structure of the wine (body, acidity, tannins, alcohol, sweetness)
            - The user's preferences, experience level, and constraints (budget, occasion, tolerance)
            Your goal is not to find a single “correct” wine, but to propose the most appropriate and enjoyable option for the given situation.

            Core Principles
            - Prioritize balance and harmony over strict pairing rules
            - Prefer safe, broadly enjoyable recommendations unless the user explicitly requests something bold or experimental
            - Avoid inventing specific brands or rare wines unless the user asks for them
            - Base all recommendations on widely accepted sommelier principles and sensory reasoning

            Communication Style
            - Use clear, friendly, and confident language
            - Avoid excessive technical jargon unless the user asks for expert-level detail
            - Briefly explain why the pairing works in an intuitive way
            - Be educational, but never patronizing

            Constraints
            - If information is missing, ask concise clarifying questions
            - Do not hallucinate unavailable data
            - If multiple good options exist, explain the difference simply

            Example 1
            User: I'm having grilled steak. Can you recommend a wine?
            Assistant:
            A medium- to full-bodied red wine with good acidity would work well. The richness of the steak pairs nicely with a wine that has enough structure, while the acidity helps balance the fat. A classic style would be a Cabernet Sauvignon or a Syrah, especially if the steak is well-seasoned or grilled.
            """
         ),
        ("human", """
            와인 페어링 추천에 아래의 요리와 풍미, 와인 리뷰만을 참고하여 한글로 답변해 주세요.

            요리와 풍미:
            {dish_flavor}
         
            와인 리뷰:
            {wine_reviews}
         
            답변은 다음과 같은 값으로 json 데이터로 리턴해주세요. 결과를 한글로 번역해 주세요.
            recommend_wine :
            recommend_reason :
        """)
    ])
    
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens=4095)
    # chain = prompt | llm | StrOutputParser()
    chain = prompt | llm | JsonOutputParser()

    return chain


In [15]:
r1 = RunnableLambda(describe_dish_flavor)
r2 = RunnableLambda(search_wines)
r3 = RunnableLambda(recommand_wines)

chain = r1 | r2 | r3
img_urls = ['https://sitem.ssgcdn.com/95/55/96/item/1000346965595_i1_750.jpg']
res = chain.invoke({
    "image_urls": img_urls
})

In [16]:
print(res)

{'recommend_wine': 'Karl Kaspar 2010 Kabinett Riesling (Mosel)', 'recommend_reason': '이 독일 모젤 지역의 리슬링은 매혹적인 향신료와 열대 과일의 풍미를 지니고 있으며, 날카롭고 강한 산미가 특징입니다. 이 와인은 매콤하고 향긋한 아시아 요리와 잘 어울리며, 김치의 매콤한 맛과 돼지고기 배의 풍부한 맛을 균형 있게 잡아줍니다.'}


In [17]:
from pprint import pprint 
pprint(res)

{'recommend_reason': '이 독일 모젤 지역의 리슬링은 매혹적인 향신료와 열대 과일의 풍미를 지니고 있으며, 날카롭고 강한 '
                     '산미가 특징입니다. 이 와인은 매콤하고 향긋한 아시아 요리와 잘 어울리며, 김치의 매콤한 맛과 '
                     '돼지고기 배의 풍부한 맛을 균형 있게 잡아줍니다.',
 'recommend_wine': 'Karl Kaspar 2010 Kabinett Riesling (Mosel)'}
